In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# import statsmodels
from scipy.stats import fisher_exact, chi2_contingency

In [6]:
# riskratio_confint
from statsmodels.stats.contingency_tables import Table2x2


In [7]:
# complete path_to_file:

path_to_file = r'D:\Ali USB Backup\Special\combined religion vs speciality (1).xlsx'

df_GMC = pd.read_excel(path_to_file, sheet_name = 'GMC data', usecols="A:J", nrows=14, index_col=0)  # non-indexed
df_HEE = pd.read_excel(path_to_file, sheet_name = 'HEE data combined', usecols="A:J", nrows=14, index_col=0)  # non-indexed


143081

In [33]:
df_HEE

,Atheism,Buddhism,Christianity,Hinduism,islam,judaism,sikh,other
Anaesthetics,307,12,239,71,70,6,9,40
Emergency medicine,300,24,283,103,259,6,10,48
GP,902,149,2060,533,1579,17,65,247
Medicine,464,177,635,226,899,20,30,91
O&G,108,24,302,93,263,6,9,29
occupational medicine,2,0,12,2,3,1,0,3
opthalmology,49,9,95,21,81,3,3,21
Paediatrics,155,19,316,76,266,6,4,40
pathology,33,2,52,13,31,2,0,13
Pschiatry,133,17,166,39,101,0,6,33


In [9]:
df_GMC

,Atheism,buddhist,christian,hindu,muslim,jewish,sikh,other,unknown and prefer not to say
anaesthesia,1278,45,1537,486,197,31,25,38,6850
Emergency Medicine,369,10,449,96,102,11,7,6,1358
GP,5225,174,7632,1364,1789,175,282,187,46649
Medicine,2116,141,3350,845,856,121,58,93,13745
Obs&gyne,334,19,806,292,190,12,11,16,2390
occupational medicine,68,0,120,9,8,2,0,1,359
opthalmology,181,14,404,105,100,9,8,12,1508
paediatrics,585,38,1120,446,204,35,9,34,3521
pathology,314,21,403,120,87,16,7,10,2035
psychiatry,936,51,1074,426,301,56,37,63,5264


# 1. Obtain the contingency matric of interest

In [34]:
def get_contingency_from_data(df, specialty, religion):
    """
    see supplementary materials. 
    in the form:
    [[a, b],
     [c, d]]
    """
    a = df.loc[specialty, religion]
    b = df[religion].sum() - a
    c = df.loc[specialty, :].sum() - a
    d = (df.sum().sum() - df.loc[specialty, :].sum()) - b
    
    contingency_table = np.array(
        [[a, b],
        [c, d]]
    )
    
    chi2, p, dof, exp = chi2_contingency(contingency_table)
    
    ct = Table2x2(contingency_table)
    RR = ct.riskratio 
    OR = ct.oddsratio
    CI = ct.riskratio_confint()
    OR_CI = ct.oddsratio_confint()
    
    print('RR', round(RR, 2), '95% CI', round(CI[0], 2), round(CI[1], 2))
    print ('chi2', round(chi2, 1), 'p', p, 'dof', dof)
    return contingency_table, chi2, p, dof, exp, OR, OR_CI

# 2. calculate RR and CI

In [11]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'Anaesthetics', 'islam')

RR 0.27 95% CI 0.21 0.34
chi2 132.4 p 1.197551979129814e-30 dof 1


In [12]:
exp

array([[  213.93293821,  4084.06706179],
       [  693.06706179, 13230.93293821]])

In [35]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'Anaesthetics', 'islam')

RR 0.26 95% CI 0.21 0.34
chi2 136.2 p 1.8054593492224566e-31 dof 1


In [36]:
exp

array([[  210.73559631,  4087.26440369],
       [  543.26440369, 10536.73559631]])

# 3. for GMC data

In [13]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_GMC, 'anaesthesia', 'muslim')

RR 0.57 95% CI 0.5 0.66
chi2 66.5 p 3.4869617706611573e-16 dof 1


In [14]:
exp

array([[   339.8649646,   4297.1350354],
       [ 10147.1350354, 128296.8649646]])

In [15]:
340-197

143

In [16]:
143/340 # deficit

0.42058823529411765

In [17]:
197/340

0.5794117647058824

# 4. HEE odds ratio as we don't know a significant portion of GMC>>HEE data religious affiliations (use especially when excluding unknowns as per 5 below)

In [37]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'Medicine', 'Buddhism')

print('\n')
print(OR)
print(OR_CI)

RR 2.29 95% CI 2.03 2.59
chi2 142.4 p 7.928505311473955e-33 dof 1


3.0341057903487347
(2.5093059733701564, 3.6686629867874356)


In [38]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'Anaesthetics', 'islam')

print('\n')
print(OR)
print(OR_CI)

RR 0.26 95% CI 0.21 0.34
chi2 136.2 p 1.8054593492224566e-31 dof 1


0.2516362650555749
(0.19626080922233297, 0.32263603794370954)


In [39]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'Medicine', 'islam')

print('\n')
print(OR)
print(OR_CI)

RR 1.41 95% CI 1.31 1.52
chi2 82.8 p 9.309816337161136e-20 dof 1


1.5191649042171116
(1.3880398575514417, 1.6626770432054792)


In [40]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'public health', 'islam')

print('\n')
print(OR)
print(OR_CI)

RR 0.26 95% CI 0.2 0.34
chi2 113.2 p 1.9839388167852967e-26 dof 1


0.2542826958778219
(0.19405594272506232, 0.3332012847166572)


In [41]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'Medicine', 'Christianity')

print('\n')
print(OR)
print(OR_CI)

RR 0.68 95% CI 0.63 0.74
chi2 85.0 p 3.00443156130636e-20 dof 1


0.6349010700470425
(0.5762645465093799, 0.6995040232625487)


In [42]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'Anaesthetics', 'Atheism')

print('\n')
print(OR)
print(OR_CI)

RR 2.74 95% CI 2.38 3.15
chi2 210.3 p 1.1726191197158346e-47 dof 1


2.933882697918834
(2.5225691392851313, 3.412262344407662)


In [43]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'Emergency medicine', 'Atheism')

print('\n')
print(OR)
print(OR_CI)

RR 1.63 95% CI 1.44 1.86
chi2 55.5 p 9.50015142562916e-14 dof 1


1.7018621291525569
(1.4786890433002775, 1.9587179060848117)


In [44]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'GP', 'Atheism')

print('\n')
print(OR)
print(OR_CI)

RR 0.77 95% CI 0.73 0.82
chi2 77.5 p 1.3396282518318915e-18 dof 1


0.6807496780115175
(0.6248184514409164, 0.7416876422968539)


In [45]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'O&G', 'Atheism')

print('\n')
print(OR)
print(OR_CI)

RR 0.59 95% CI 0.49 0.72
chi2 27.2 p 1.8657778642277284e-07 dof 1


0.5789794097288823
(0.4711372202790047, 0.7115064199162439)


In [46]:
contingency_table, chi2, p, dof, exp, OR, OR_CI = get_contingency_from_data(df_HEE, 'public health', 'Atheism')

print('\n')
print(OR)
print(OR_CI)

RR 1.79 95% CI 1.52 2.1
chi2 48.2 p 3.947848739809637e-12 dof 1


1.8394340115381875
(1.5467874086501527, 2.1874482969551576)


# 5. ok so above uses unkowns, we now need to check the magnitude of these effects with sensitivity analyses

In [32]:
# simply run this then run above again:
df_HEE.drop(columns=['unknown'], inplace=True)